In [1]:
# Your code here
import gymnasium as gym
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from torch import optim
from torch.distributions import Categorical

class Policy(nn.Module):
    def __init__(self, env):
        super(Policy, self).__init__()
        self.n_inputs = env.observation_space.shape[0]
        self.n_outputs = env.action_space.n
        
        self.affine1 = nn.Linear(self.n_inputs, 128)
        self.dropout = nn.Dropout(p=0.6)
        self.affine2 = nn.Linear(128, self.n_outputs)

        self.saved_log_probs = []
        self.rewards = []

    def forward(self, x):
        x = self.affine1(x)
        x = self.dropout(x)
        x = F.relu(x)
        action_scores = self.affine2(x)
        return F.softmax(action_scores, dim=1)
    
    def select_action(self, state):
        state = torch.from_numpy(np.array(state)).float().unsqueeze(0)
        probs = self.forward(state)
        m = torch.distributions.Categorical(probs)
        action = m.sample()
        self.saved_log_probs.append(m.log_prob(action))
        return action.item()


c:\Python3.10.4\lib\site-packages\gymnasium\envs\registration.py:693: UserWarning: WARN: Overriding environment GymV26Environment-v0 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")
c:\Python3.10.4\lib\site-packages\gymnasium\envs\registration.py:604: UserWarning: WARN: plugin: shimmy.registration:register_gymnasium_envs raised Traceback (most recent call last):
  File "c:\Python3.10.4\lib\site-packages\gymnasium\envs\registration.py", line 602, in load_plugin_envs
    fn()
  File "c:\Python3.10.4\lib\site-packages\shimmy\registration.py", line 256, in register_gymnasium_envs
    _register_dm_control_envs()
  File "c:\Python3.10.4\lib\site-packages\shimmy\registration.py", line 26, in _register_dm_control_envs
    from shimmy.dm_control_compatibility import DmControlCompatibilityV0
  File "c:\Python3.10.4\lib\site-packages\shimmy\dm_control_compatibility.py", line 15, in <module>
    from dm_control import composer
  File "c:\Python3.10.4

In [2]:
#RL environment parameters
gamma = 0.95
seed = 0
render = False
log_interval = 10

#Set up environment
env = gym.make("LunarLander-v2", render_mode="human")
# env = gym.make("CartPole-v1", render_mode='rgb_array')
env.reset(seed=seed)
torch.manual_seed(seed)

#Create out policy Network
policy = Policy(env)
optimizer = optim.Adam(policy.parameters(), lr=1e-2)
eps = np.finfo(np.float32).eps.item()

is_done = False
state, info = env.reset()

In [3]:
while not is_done:
    action = policy.select_action(state)
    
    state, reward, done, truncated, info = env.step(action)
    # print(reward, is_done)
    screen = env.render()
env.close()

KeyboardInterrupt: 